# Brill-Lindquist Initial data

## This module sets up initial data for a merging black hole system in Cartesian coordinates

<font color='green'>**This module has been validated to exhibit convergence to zero of the Hamiltonian constraint violation at the expected order to the exact solution (see plot at bottom), and all quantities have been validated against the [original SENR code](https://bitbucket.org/zach_etienne/nrpy).**</font>

### Here we set up Brill-Lindquist initial data ([Brill & Lindquist, Phys. Rev. 131, 471, 1963](https://journals.aps.org/pr/abstract/10.1103/PhysRev.131.471); see also Eq. 1 of [Brandt & Brügmann, arXiv:gr-qc/9711015v1](https://arxiv.org/pdf/gr-qc/9711015v1.pdf)):

The conformal factor $\psi$ for Brill-Lindquist initial data is given by
$$\psi = e^{\phi} = 1 + \sum_{i=1}^N \frac{m_{(i)}}{2 \left|\vec{r}_{(i)} - \vec{r}\right|};\quad K_{ij}=0,$$

where $\psi$ is written in terms of the 3-metric $\gamma_{ij}$ as

$$
\gamma_{ij} = \psi^4 \delta_{ij}.
$$

The extrinsic curvature is zero:
$$
K_{ij} = 0
$$

These data consist of $N$ nonspinning black holes initially at rest. This module restricts to the case of two such black holes, positioned anywhere. Here, we implement $N=2$.

**Inputs for $\psi$**:
* The position and (bare) mass of black hole 1: $\left(x_{(1)},y_{(1)},z_{(1)}\right)$ and $m_{(1)}$, respectively
* The position and (bare) mass of black hole 2: $\left(x_{(2)},y_{(2)},z_{(2)}\right)$ and $m_{(2)}$, respectively

**Additional variables needed for spacetime evolution**:
* Desired coordinate system
* Desired initial lapse $\alpha$ and shift $\beta^i$. We will choose our gauge conditions as $\alpha=\psi^{-2}$ and $\beta^i=B^i=0$.

In [1]:
# Step P0: Load needed modules
import sympy as sp
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
from outputC import *

thismodule = "Brill-Lindquist"
BH1_posn_x,BH1_posn_y,BH1_posn_z = par.Cparameters("REAL", thismodule, ["BH1_posn_x","BH1_posn_y","BH1_posn_z"])
BH1_mass = par.Cparameters("REAL", thismodule, ["BH1_mass"])
BH2_posn_x,BH2_posn_y,BH2_posn_z = par.Cparameters("REAL", thismodule, ["BH2_posn_x","BH2_posn_y","BH2_posn_z"])
BH2_mass = par.Cparameters("REAL", thismodule, ["BH2_mass"])

# Step 0: Set spatial dimension (must be 3 for BSSN)
DIM = 3
par.set_parval_from_str("grid::DIM",DIM)

Cartxyz = ixp.declarerank1("Cartxyz")

# Step 1: Set psi, the conformal factor:
psi = sp.sympify(1)
psi += BH1_mass / ( 2 * sp.sqrt((Cartxyz[0]-BH1_posn_x)**2 + (Cartxyz[1]-BH1_posn_y)**2 + (Cartxyz[2]-BH1_posn_z)**2) )
psi += BH2_mass / ( 2 * sp.sqrt((Cartxyz[0]-BH2_posn_x)**2 + (Cartxyz[1]-BH2_posn_y)**2 + (Cartxyz[2]-BH2_posn_z)**2) )
    
# Step 2: Set all needed ADM variables in Cartesian coordinates
gammaCartDD = ixp.zerorank2()
KCartDD     = ixp.zerorank2() # K_{ij} = 0 for these initial data
for i in range(DIM):
    gammaCartDD[i][i] = psi**4

alphaCart = 1/psi**2
betaCartU = ixp.zerorank1() # We generally choose \beta^i = 0 for these initial data
BCartU    = ixp.zerorank1() # We generally choose B^i = 0 for these initial data

## NRPy+ Module Code Validation

Here, as a code validation check, we verify agreement in the SymPy expressions for Brill-Lindquist initial data between

1. this tutorial and 
2. the NRPy+ [BSSN.BrillLindquist](../edit/BSSN/BrillLindquist.py) module.

In [2]:
# Reset the list of gridfunctions, as registering a gridfunction
#   twice will spawn an error.
gri.glb_gridfcs_list = []

# First we import needed core NRPy+ modules
import reference_metric as rfm
import grid as gri
# Unused, but needed because BrillLindquist() also does tensor rescalings.
rfm.reference_metric()

import BSSN.BrillLindquist as bl
bl.BrillLindquist()

# It is SAFE to ignore the warning(s) from re-initializing parameters.
print("^^^ Ignore the minor warning(s) above. ^^^\n")
print("Consistency check between Brill-Lindquist tutorial and NRPy+ BSSN.BrillLindquist module:")
print("alphaCart - bl.alphaCart: Should be zero: "+str(sp.simplify(alphaCart - bl.alphaCart)))
for i in range(DIM):
    print("betaCartU["+str(i)+"] - bl.betaCartU["+str(i)+"]: Should be zero: "+\
          str(sp.simplify(betaCartU[i] - bl.betaCartU[i])))
    print("BCartU["+str(i)+"] - bl.BaCartU["+str(i)+"]: Should be zero: "+str(sp.simplify(BCartU[i] - bl.BCartU[i])))
    for j in range(DIM):
        print("gammaCartDD["+str(i)+"]["+str(j)+"] - bl.gammaCartDD["+str(i)+"]["+str(j)+"]: Should be zero: "+\
              str(sp.simplify(gammaCartDD[i][j] - bl.gammaCartDD[i][j])))
        print("KCartDD["+str(i)+"]["+str(j)+"] - bl.KCartDD["+str(i)+"]["+str(j)+"]: Should be zero: "+\
              str(sp.simplify(KCartDD[i][j] - bl.KCartDD[i][j])))

initialize_param() minor warning: Did nothing; already initialized parameter Brill-Lindquist::BH1_posn_x
initialize_param() minor warning: Did nothing; already initialized parameter Brill-Lindquist::BH1_posn_y
initialize_param() minor warning: Did nothing; already initialized parameter Brill-Lindquist::BH1_posn_z
initialize_param() minor warning: Did nothing; already initialized parameter Brill-Lindquist::BH1_mass
initialize_param() minor warning: Did nothing; already initialized parameter Brill-Lindquist::BH2_posn_x
initialize_param() minor warning: Did nothing; already initialized parameter Brill-Lindquist::BH2_posn_y
initialize_param() minor warning: Did nothing; already initialized parameter Brill-Lindquist::BH2_posn_z
initialize_param() minor warning: Did nothing; already initialized parameter Brill-Lindquist::BH2_mass
^^^ Ignore the minor warning(s) above. ^^^

Consistency check between Brill-Lindquist tutorial and NRPy+ BSSN.BrillLindquist module:
alphaCart - bl.alphaCart: Shoul